<a href="https://colab.research.google.com/github/SebastianLarssonDTU/02456-Reinforcement-Learning-Project/blob/before_framestacking/Generate_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INIT : Procgen, Drive, Git

In [1]:
!pip install procgen

     |████████████████████████████████| 39.9MB 83kB/s 
     |████████████████████████████████| 51kB 8.7MB/s 
     |████████████████████████████████| 3.3MB 53.8MB/s 
     |████████████████████████████████| 665kB 50.5MB/s 
     |████████████████████████████████| 204kB 55.1MB/s 
     |████████████████████████████████| 22.2MB 1.5MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1


In [2]:
#Clone git
!git clone -b before_framestacking https://github.com/SebastianLarssonDTU/02456-Reinforcement-Learning-Project.git "my_project"

Cloning into 'my_project'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (164/164), done.
remote: Total 624 (delta 99), reused 4 (delta 0), pack-reused 460
Receiving objects: 100% (624/624), 12.08 MiB | 16.81 MiB/s, done.
Resolving deltas: 100% (382/382), done.


In [3]:
#update git
%cd /content/my_project
! git pull

/content/my_project
Already up to date.


In [4]:
import datatools as tools
from datatools import DATA_PATH, MODEL_PATH
#Mount drive
tools.mount_drive()

Mounted at /content/drive


# TESTING

In [5]:
#Import all custom files
import baseline
import datatools as tools
import hyperparameters as h
import model
import my_util
import policy
import ppo
import utils

#other imports
import torch

#import specific methods
from baseline import set_hyperparameters
from ppo import PPO
from experiments import run_experiment, print_list_of_experiments


In [6]:
# run_experiment(2, levels=50, save_interval=5e5)

In [7]:
# #for Experiment 7
# file_name = "Experiment7_50levels_Run_04Dec_18h12m45s_loaded_05Dec_10h49m07s_13516800steps"
# set_hyperparameters(baseline="Impala")
# h.value_clipping = True
# h.num_levels = 50
# h.batch_size=512
# model = PPO(print_output=True, eval=True, save_interval=5e5)
# model.load_policy(file_name)
# model.train()

In [8]:
# #For Experiment 2
# file_name = "Experiment2_50levels_Run_04Dec_18h10m32s_loaded_05Dec_09h43m36s_13516800steps"
# set_hyperparameters(baseline="PPO")
# h.value_clipping = True
# h.num_levels = 50
# model = PPO(print_output=True, eval=True, save_interval=5e5)
# model.load_policy(file_name)
# model.train()

In [9]:
# #for Experiment 8
# file_name = "Experiment8_50levels_Run_05Dec_08h34m02s_4005888steps"
# set_hyperparameters(baseline="Impala")
# h.value_clipping = True
# h.num_levels = 50
# h.batch_size=512
# h.death_penalty = True
# h.penalty = 5
# model = PPO(print_output=True, eval=True, save_interval=5e5)
# model.load_policy(file_name)
# model.train()

# Generate video

In [10]:
from utils import make_env
import imageio

def old_policy_evaluation(model, video_name=None, print_output=True, test_on_training=False, off_set = 0):
  policy = model.policy

  if video_name is None:
    video_name = model.file_name
  # Make evaluation environment
  if test_on_training:
    eval_env = make_env(model.num_envs, num_levels=model.num_levels)
    video_name += "_TEST_ON_TRAINING"
  else:
    eval_env = make_env(model.num_envs, start_level=model.num_levels+off_set, num_levels=model.num_levels)
  obs = eval_env.reset()

  frames = []
  total_reward = []

  # Evaluate policy
  policy.eval()
  # for _ in range(512):  #<--- Hardcoded batch size?
  while True:
    # Use policy
    action, log_prob, value = policy.act(obs)

    # Take step in environment
    obs, reward, done, info = eval_env.step(action)
    total_reward.append(torch.Tensor(reward))

    # Render environment and store
    frame = (torch.Tensor(eval_env.render(mode='rgb_array'))*255.).byte()
    frames.append(frame)
    if done[0]:
      break

  # Calculate average return
  total_reward = torch.stack(total_reward).sum(0).mean(0)
  
  if print_output:
    print('Average return:', total_reward)

  # Save frames as video
  frames = torch.stack(frames).cpu().numpy()
  imageio.mimsave(video_name+'.mp4', frames, fps=25)
  
  return total_reward

## New code

In [41]:
from utils import make_env
import imageio

def generate_videos(model, video_name=None, print_output=True, test_on_training=False, off_set = 0, nr_of_levels=5, save_video=False):
  policy = model.policy

  if video_name is None:
    video_name = model.file_name
  
  # Make evaluation environment
  if test_on_training:
    eval_env = make_env(model.num_envs, num_levels=nr_of_levels, normalize_reward = False)
    video_name += "_TEST_ON_TRAINING"
  else:
    eval_env = make_env(model.num_envs, start_level=model.num_levels+off_set, num_levels=nr_of_levels, normalize_reward = False)
  obs = eval_env.reset()

  frames = []
  total_reward = []
  level_counter=1
  # Evaluate policy
  policy.eval()

  while True:
    
    # Use policy
    action, log_prob, value = policy.act(obs)

    # Take step in environment
    obs, reward, done, info = eval_env.step(action)
    total_reward.append(torch.Tensor(reward))

    # Render environment and store
    frame = (torch.Tensor(eval_env.render(mode='rgb_array'))*255.).byte()
    frames.append(frame)
    if done[0]: #next level
      #Calculate level reward
      level_reward = torch.stack(total_reward).sum(0)[0]
      
      if save_video:
        # Save frames as video
        frames = torch.stack(frames).cpu().numpy()
        name = video_name+'_level{:d}_reward{:.0f}.mp4'.format(level_counter, level_reward)
        imageio.mimsave(name, frames, fps=25)
        print("Saved video: {}".format(name))
      else:
        print(level_counter)
        print(level_reward)
      
      #check if all levels done
      level_counter +=1
      if level_counter > nr_of_levels:
        break #done testing

      #reset variables for next level
      frames=[]
      total_reward = []

## Load model

In [18]:
#for Experiment 8
file_name = "Experiment8_200levels_Run_06Dec_16h17m59s_loaded_08Dec_10h26m35s_5021696steps"
set_hyperparameters(baseline="Impala")
h.value_clipping = True
h.num_levels = 200
h.batch_size=512
h.death_penalty = True
h.penalty = 5
model = PPO(print_output=True, eval=True, save_interval=5e5)
model.load_policy(file_name)

Observation space: Box(0.0, 1.0, (3, 64, 64), float32)
Action space: 15
Loaded current model from models folder with name Experiment8_200levels_Run_06Dec_16h17m59s_loaded_08Dec_10h26m35s_5021696steps.pt


Policy(
  (encoder): ImpalaEncoder(
    (block1): ImpalaBlock(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (res1): ResidualBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (res2): ResidualBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (block2): ImpalaBlock(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (res1): ResidualBlock(
        (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (res2): ResidualBlock(
        (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(

In [12]:
# old_policy_evaluation(model, test_on_training=True)

In [21]:
model.evaluate_policy(5, normalize_reward=False)

(33.0125,
 {0: [39.0, 28.0, 19.0, 32.0, 7.0],
  1: [6.0, 17.0, 7.0, 36.0, 39.0],
  2: [62.0, 69.0, 51.0, 4.0, 14.0],
  3: [39.0, 21.0, 5.0, 9.0, 59.0],
  4: [7.0, 35.0, 5.0, 21.0, 15.0],
  5: [10.0, 7.0, 49.0, 29.0, 62.0],
  6: [34.0, 3.0, 17.0, 4.0, 68.0],
  7: [64.0, 47.0, 52.0, 61.0, 4.0],
  8: [7.0, 68.0, 7.0, 59.0, 45.0],
  9: [57.0, 13.0, 56.0, 37.0, 6.0],
  10: [15.0, 38.0, 38.0, 58.0, 21.0],
  11: [70.0, 50.0, 2.0, 65.0, 69.0],
  12: [6.0, 6.0, 56.0, 58.0, 5.0],
  13: [62.0, 4.0, 56.0, 12.0, 43.0],
  14: [41.0, 51.0, 70.0, 68.0, 17.0],
  15: [7.0, 10.0, 14.0, 15.0, 56.0],
  16: [59.0, 45.0, 57.0, 64.0, 16.0],
  17: [67.0, 67.0, 47.0, 66.0, 37.0],
  18: [38.0, 4.0, 8.0, 14.0, 19.0],
  19: [8.0, 53.0, 68.0, 55.0, 5.0],
  20: [71.0, 62.0, 10.0, 59.0, 52.0],
  21: [46.0, 33.0, 7.0, 58.0, 66.0],
  22: [14.0, 15.0, 63.0, 14.0, 29.0],
  23: [29.0, 7.0, 18.0, 10.0, 32.0],
  24: [13.0, 49.0, 5.0, 59.0, 4.0],
  25: [26.0, 56.0, 7.0, 9.0, 65.0],
  26: [17.0, 22.0, 65.0, 39.0, 17.0],
  27:

In [22]:
model.evaluate_policy(5, normalize_reward=True)

(9.602030308544636,
 {0: [21.66693340241909,
   6.668309599161148,
   4.558768928050995,
   7.732477232813835,
   1.7077887654304504],
  1: [15.279632449150085,
   4.842270344495773,
   1.6627369225025177,
   8.568100154399872,
   9.482200741767883],
  2: [18.68360722064972,
   16.605701461434364,
   12.543238922953606,
   0.9899160414934158,
   3.4640539586544037],
  3: [24.464234605431557,
   4.977383449673653,
   1.1971215158700943,
   2.1559195816516876,
   14.355506390333176],
  4: [16.490240216255188,
   9.38219590485096,
   1.1791674196720123,
   5.01148396730423,
   3.6015762239694595],
  5: [6.894734710454941,
   2.0540365874767303,
   11.68256014585495,
   6.975977569818497,
   15.218755930662155],
  6: [25.987394779920578,
   0.6983423382043839,
   4.0503754913806915,
   0.9580339789390564,
   16.429210543632507],
  7: [22.784125417470932,
   11.250742793083191,
   12.675657376646996,
   15.099013596773148,
   0.985857218503952],
  8: [9.350831896066666,
   16.97480462491512

In [40]:
generate_videos(model, video_name="Impala_dp5_200levels_test10", save_video=True, nr_of_levels=10)

Saved video: Impala_dp5_200levels_test10_level1_reward39.mp4
Saved video: Impala_dp5_200levels_test10_level2_reward16.mp4
Saved video: Impala_dp5_200levels_test10_level3_reward37.mp4
Saved video: Impala_dp5_200levels_test10_level4_reward66.mp4
Saved video: Impala_dp5_200levels_test10_level5_reward23.mp4


### Impala no penalty 200 levels

In [13]:
# #for Experiment 7
# file_name = "Experiment7_200levels_Run_06Dec_16h18m31s_loaded_07Dec_11h04m01s_7012352steps"
# set_hyperparameters(baseline="Impala")
# h.value_clipping = True
# h.num_levels = 200
# h.batch_size=512
# model = PPO(print_output=True, eval=True, save_interval=5e5)
# model.load_policy(file_name)

In [14]:
# old_policy_evaluation(model)

### PPO 200 levels

In [15]:
# #For Experiment 2
# file_name = "Experiment2_200levels_Run_07Dec_13h49m13s_loaded_08Dec_00h03m23s_7012352steps"
# set_hyperparameters(baseline="PPO")
# h.value_clipping = True
# h.num_levels = 200
# model = PPO(print_output=True, eval=True, save_interval=5e5)
# policy=model.load_policy(file_name)

In [16]:
# for i in range(4):
#   result=old_policy_evaluation(model, video_name="PPO_200levels_{}".format(i), off_set=i, print_output=False)
#   print("{}: {}".format(i, result))
  

In [17]:
# old_policy_evaluation(model, test_on_training=True)